# Final project: Nólë 
by Débora Machado Andrade

**Nólë** means "knowledge" in Quenya, a fictional Elvish language created by J.R.R.Tolkien 

In the following notebook we'll build RAG and agentic pipelines that will allow us to interactively retrieve information from our knowledge sources, as well as construct the building blocks of a narrative story-telling book for children, where the chosen knowledge will be embedded. 

## Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [1]:
import os
import openai
from openai import AsyncOpenAI  # importing openai for API usage
import chainlit as cl  # importing chainlit for our app
from chainlit.prompt import Prompt, PromptMessage  # importing prompt tools
from chainlit.playground.providers import ChatOpenAI  # importing ChatOpenAI tools

from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key



In [5]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, load_tools

llm = ChatOpenAI(model_name="gpt-4", temperature=0)

tools = load_tools(["dalle-image-generator"])
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
output = agent.run("Create an image of Seb, Deb, Becsy, Ceci and Tonton having dinner at a rectangular table. Seb is a 38 year old tall man that wears a t-shirt, has reddish light brown hair and a red short beard; Deb is a 39 year old woman, dark wavy hair shoulder length, Becsy is a 7 year-old girl with long flat brown hair, Ceci is a 2 year old girl with flat blond hair shoulder length, and Tonton is an identical twin to Ceci (so they look the same, but wear different clothes).")



> Entering new AgentExecutor chain...
2024-03-23 15:48:02 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
I need to generate an image based on the provided description. The Dall-E-Image-Generator tool can be used for this purpose.
Action: Dall-E-Image-Generator
Action Input: "Seb, a 38 year old tall man with reddish light brown hair and a red short beard, wearing a t-shirt, Deb, a 39 year old woman with dark wavy hair shoulder length, Becsy, a 7 year-old girl with long flat brown hair, Ceci and Tonton, 2 year old identical twin girls with flat blond hair shoulder length, wearing different clothes, having dinner at a rectangular table. The style should be similar to Sir John Tenniel."2024-03-23 15:48:15 - HTTP Request: POST https://api.openai.com/v1/images/generations "HTTP/1.1 200 OK"

Observation: https://oaidalleapiprodscus.blob.core.windows.net/private/org-3ehqwsuCdbx8j7bzKXYPD2wu/user-LWJG0KwqXtTsrDOPSvUvW3S0/img-89SDkojcxuOWjXJBNwBZpqNT.png?st=2

## Building our RAG pipeline

#### Loading Data

In [88]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "data/Tratado_Descritivo_Brasil_1587.pdf",
)

documents = loader.load()

In [89]:
documents[0].metadata

{'source': 'data/NVIDIA_report.pdf',
 'file_path': 'data/NVIDIA_report.pdf',
 'page': 0,
 'total_pages': 96,
 'format': 'PDF 1.4',
 'title': '0001045810-24-000029',
 'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group',
 'subject': 'Form 10-K filed on 2024-02-21 for the period ending 2024-01-28',
 'keywords': '0001045810-24-000029; ; 10-K',
 'creator': 'EDGAR Filing HTML Converter',
 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0',
 'creationDate': "D:20240221173732-05'00'",
 'modDate': "D:20240221173744-05'00'",
 'trapped': '',
 'encryption': 'Standard V2 R3 128-bit RC4'}

In [90]:
len(documents)

96

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [91]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [92]:
len(documents)

438

In [93]:
documents[1]

Document(page_content='Title of each class\nTrading Symbol(s)\nName of each exchange on which registered\nCommon Stock, $0.001 par value per share\nNVDA\nThe Nasdaq Global Select Market\nSecurities registered pursuant to Section 12(g) of the Act:\nNone\nIndicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act.    Yes ☐ No ☒\nIndicate by check mark if the registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act.    Yes ☐ No ☒\nIndicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during the preceding 12 months (or for such shorter\nperiod that the registrant was required to file such reports), and (2) has been subject to such filing requirements for the past 90 days. Yes ☒ No ☐', metadata={'source': 'data/NVIDIA_report.pdf', 'file_path': 'data/NVIDIA_report.pdf', 'page': 0, 'total_pages': 9

#### Loading OpenAI Embeddings Model

We will use use OpenAI's `text-embedding-3-small` for this task.

In [94]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

#### Creating a FAISS VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [95]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

2024-03-13 18:24:10 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever:

In [96]:
retriever = vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [97]:
retrieved_documents = retriever.invoke("What is this document about?")

2024-03-13 18:24:13 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [98]:
for doc in retrieved_documents:
  print(doc)

page_content='23.1*\nConsent of PricewaterhouseCoopers LLP\n24.1*\nPower of Attorney (included in signature page)\n31.1*\nCertification of Chief Executive Officer as required by Rule 13a-14(a) of the Securities Exchange Act of 1934\n31.2*\nCertification of Chief Financial Officer as required by Rule 13a-14(a) of the Securities Exchange Act of 1934\n32.1#*\nCertification of Chief Executive Officer as required by Rule 13a-14(b) of the Securities Exchange Act of 1934\n32.2#*\nCertification of Chief Financial Officer as required by Rule 13a-14(b) of the Securities Exchange Act of 1934\n97.1+*\nCompensation Recovery Policy, as amended and restated November 30, 2023\n101.INS*\nXBRL Instance Document\n101.SCH*\nXBRL Taxonomy Extension Schema Document\n101.CAL*\nXBRL Taxonomy Extension Calculation Linkbase Document\n101.DEF*\nXBRL Taxonomy Extension Definition Linkbase Document\n101.LAB*\nXBRL Taxonomy Extension Labels Linkbase Document\n101.PRE*\nXBRL Taxonomy Extension Presentation Linkbase 

### Creating a RAG Chain


#### Creating a Prompt Template


In [164]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [100]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

Above we have a RAG chain that first uses Python's itemgetter to extract the "question" from input, passing it to a retriever but also keeping the original "question" intact. A RunnablePassthrough then temporarily holds the "context" (which is obtained as an output of the "question" chained into the retriever) without altering it. Finally, the "context" and "question" are used as inputs for a prompt for ChatOpenAI, generating a "response".

Let's test it out!

In [160]:
question = "What is the provided document about?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

2024-03-13 19:37:45 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-13 19:37:46 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
I don't know.


In [102]:
question = "Who is the E-VP, Operations - and how old are they?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

2024-03-13 18:24:15 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-13 18:24:16 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Debora Shoquist is the Executive Vice President of Operations, and she is 69 years old.
[Document(page_content='Minnesota, an M.S.E.E. degree from the California Institute of Technology and an M.B.A. degree from Harvard Business School.\nDebora Shoquist joined NVIDIA in 2007 as Senior Vice President of Operations and in 2009 became Executive Vice President of Operations. Prior to NVIDIA,\nMs. Shoquist served from 2004 to 2007 as Executive Vice President of Operations at JDS Uniphase Corp., a provider of communications test and measurement\nsolutions and optical products for the telecommunications industry. She served from 2002 to 2004 as Senior Vice President and General Manager of the Electro-\nOptics business at Coherent, Inc., a manufacturer of commercial and scientific laser equipment. P

## Making Adjustments to our RAG Pipeline

Now we upgrade our retriever. We use the MultiQueryRetriever which is the way to go:

In [165]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0

First, let's create a chain to "stuff" our documents into our context:

In [173]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain:

In [174]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [175]:
response = retrieval_chain.invoke({"input": "What is the provided document about?"})

2024-03-13 20:04:07 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-13 20:04:07 - Generated queries: ['1. Can you summarize the content of the document?', '2. What information does the document contain?', "3. Could you give me an overview of the document's main topics?"]
2024-03-13 20:04:07 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-13 20:04:07 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-13 20:04:08 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-13 20:04:09 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [176]:
print(response["answer"])

The provided document is an Annual Report on Form 10-K for NVIDIA Corporation. It includes various sections such as Business, Risk Factors, Cybersecurity, Market for Registrant’s Common Equity, Financial Statements, Directors and Executive Officers, Executive Compensation, and other relevant information required by the Securities and Exchange Commission (SEC).


In [177]:
response = retrieval_chain.invoke({"input": "What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023?"})

2024-03-13 20:04:12 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-13 20:04:12 - Generated queries: ['1. How much is the total carrying value of Amortizable Intangible Assets as of January 29, 2023?', '2. What is the total gross amount of Amortizable Intangible Assets that can be amortized as of January 29, 2023?', '3. Can you provide the total value of Amortizable Intangible Assets that are eligible for amortization on January 29, 2023?']
2024-03-13 20:04:12 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-13 20:04:12 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-13 20:04:13 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-13 20:04:14 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [178]:
print(response["answer"])

The gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023, was $3,539 million.
